In [ ]:
import csv
import pandas as pd
import numpy as np
import dask.dataframe as da
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider1 = PlainTextAuthProvider(username='orderhub_test', password='2345/*')
archival_cluster = Cluster(['u060dcxyz'],port = 8142,auth_provider=auth_provider1)
session = archival_cluster.connect('orders')
#warnings.filterwarnings("ignore")

files = 1
end_file = 8
directory= 'apps/python-docker/comparing'

while (files <= end_file):
        
    try:
        
        df_read_backup=da.read_csv(f'{directory}/validating/mark_validation_{files}.csv')
        df_read_backup['count'] = df_read_backup['count'].astype(int) 
               
        order_count = list(df_read_backup['count'].astype(int))
        order_list = list(df_read_backup['order_id'])                            
        order_list1 = ', '.join(f"'{orderno}'" for orderno in order_list)        
        #print('order_str_list', order_str_list)
        
        query1 = "select count(1) as count_db, order_id from orders.kcp_events where order_id in (%s) group by order_id"%order_list1
        #query1 = "select count(1), order_id from orders.kcp_events where order_id in ('%s')"%order_str_list
        query_result = session.execute(query1)              
        df_query_result = pd.DataFrame(query_result)        
        df_query_result['order_id'] = df_query_result['order_id'].astype(np.int64)        
        df_query_result['count_db'] = df_query_result['count_db'].astype(int)  
 
        df_combine = pd.merge(df_read_backup.compute(), df_query_result, how="left", on=['order_id'])
        print(df_combine.head(15))
        
        df_combine['isValidCount'] = (df_combine['count_db'] == (df_combine['count']))
        #print(df_combine.head())
        
        # #write in csv validation file
        df_combine.loc[(df_combine['isValidCount'] == True )][['order_id']]\
            .to_csv(f'{directory}/comparing/mark_deletion_{files}.csv' , index = False)
            
        ##write non-validation file
        df_combine.loc[(df_combine['isValidCount'] == False )]['order_id']\
            .to_csv(f'{directory}/comparing/Not_deletion_{files}.csv' , index = False)
      
        print('No of completed files:',files) 
        files += 1


    except Exception as e:
        print (e) 
        files += 1
